**Personalized AI Tutor for Diploma-Level Students**

This cell enables core features required for building an adaptive AI tutoring system.
CSV handling allows the system to load question data externally instead of hardcoding it.
Randomization supports fair and non-repetitive question selection.


In [38]:
import csv, random, time, os, ast


This cell adds configurability to the system.
Passing score, file paths, and levels can be changed without touching logic.
This improves maintainability.This cell introduces configurability into the system design.
Passing criteria, file paths, and learning levels are defined in one place.
This allows educators to modify evaluation rules without changing core logic.
It separates policy decisions from implementation.
Such separation improves maintainability and reuse.

In [39]:
BASE_DIR = os.getcwd()
QUESTIONS_CSV = os.path.join(BASE_DIR, "IIT final project data.csv")
PROGRESS_FILE = os.path.join(BASE_DIR, "progress.csv")

TOPIC_PASS_PERCENT = 60

LEVELS = [
    ("Beginner", 0),
    ("Learner", 100),
    ("Skilled", 250),
    ("Advanced", 500),
    ("Pro", 800),
]


This feature makes the system robust to different dataset formats.
It avoids strict dependency on exact column names in the CSV file.
This allows reuse of the system with datasets from different sources.
It reduces data preparation errors during deployment.
Such flexibility is critical in real educational environments.
This improves system reliability

In [40]:
def _find_column_map(fieldnames):
    hdrs = {h.lower().strip(): h for h in (fieldnames or [])}
    def col(*names):
        for n in names:
            if n.lower() in hdrs:
                return hdrs[n.lower()]
        return None
    return col


This cell handles data understanding and preparation.
Questions are loaded dynamically from a CSV file and grouped by topic.
Basic and advanced questions are separated automatically.
This design supports content scalability without code changes.
It allows instructors to update or expand content easily.
This aligns with data-driven AI system design.

In [41]:
def load_questions_from_csv(path):
    if not os.path.exists(path):
        print("CSV not found:", path)
        return None

    basics = {}
    advanced = {}
    topics_meta = {}
    topicid_by_name = {}
    order = []
    next_id = 1

    with open(path, newline="", encoding="utf-8-sig") as f:
        reader = csv.DictReader(f)
        col = _find_column_map(reader.fieldnames)

        C_SEC = col("Section", "Type")
        C_TOPIC = col("Topic")
        C_TID = col("Topic_ID", "TopicID")
        C_LVL = col("Level")
        C_Q = col("Question")
        C_A = col("Option A", "A")
        C_B = col("Option B", "B")
        C_C = col("Option C", "C")
        C_D = col("Option D", "D")
        C_COR = col("Correct", "Answer")
        C_EXP = col("Explanation")

        for row in reader:
            sec = (row.get(C_SEC) or "").lower()
            topic = (row.get(C_TOPIC) or "").strip()
            if not topic:
                continue

            tid_raw = row.get(C_TID)
            if tid_raw:
                try:
                    tid = int(tid_raw)
                except:
                    continue
            else:
                if topic not in topicid_by_name:
                    topicid_by_name[topic] = next_id
                    next_id += 1
                tid = topicid_by_name[topic]

            if tid not in order:
                order.append(tid)
            topics_meta[tid] = topic

            opts = [
                (row.get(C_A) or "").strip(),
                (row.get(C_B) or "").strip(),
                (row.get(C_C) or "").strip(),
                (row.get(C_D) or "").strip()
            ]
            ans = (row.get(C_COR) or "").strip().lower()
            exp = (row.get(C_EXP) or "").strip()

            if not all(opts) or ans not in ("a","b","c","d"):
                continue

            if sec.startswith("basic"):
                basics.setdefault(tid, []).append((row[C_Q], opts, ans, exp))
            elif sec.startswith("adv"):
                lvl = (row.get(C_LVL) or "easy").lower()
                if lvl not in ("easy","medium","hard"):
                    lvl = "easy"
                advanced.setdefault(tid, {"easy":[], "medium":[], "hard":[]})
                advanced[tid][lvl].append((row[C_Q], opts, ans, exp))

    return order, topics_meta, basics, advanced


This feature manages how learners interact with the system.
It ensures that only valid inputs are accepted during quizzes.
This prevents crashes and incorrect evaluations.
It improves the overall user experience.
The interaction is designed to be simple and learner-friendly.
This is essential for diploma-level usability.

In [42]:
def ask_q(q, opts):
    print("\nQ:", q)
    for i, o in enumerate(opts):
        print(f"  {chr(97+i)}) {o}")
    while True:
        a = input("Your answer (a/b/c/d or exit): ").strip().lower()
        if a == "exit":
            return "exit"
        if a in ("a","b","c","d"):
            return a


This cell introduces topic-wise assessment instead of a single global test.
Each topic is evaluated independently to identify learning gaps.
This prevents students from progressing with weak foundations.
It supports mastery-based learning.
Scores from this stage influence later system behavior.
This mimics real adaptive learning platforms.

In [43]:
def run_basic_topic(qs, topic, idx):
    print("\n" + "="*60)
    print(f"BASIC TOPIC {idx}: {topic}")
    print("="*60)

    correct = 0
    total = len(qs)

    for q, opts, ans, exp in qs:
        user = ask_q(q, opts)
        if user == "exit":
            return 0

        if user == ans:
            print("Correct")
            correct += 1
        else:
            print(f"Wrong | Correct: {ans}) {opts[ord(ans)-97]}")
            if exp:
                print(exp)

    return (correct / total) * 100 if total else 0


This cell implements adaptive difficulty progression.
Question difficulty increases or decreases based on learner performance.
Strong learners are challenged more, while weaker learners get support.
This avoids both boredom and overload.
Such adaptation is a core AI application concept.
It personalizes learning without using complex ML models.

In [44]:
def run_advanced_topic(topic, pool):
    print("\nADVANCED MODE:", topic)
    levels = ["easy","medium","hard"]
    idx = 0
    streak = 0
    asked = {l:set() for l in levels}

    while True:
        lvl = levels[idx]
        if lvl not in pool or not pool[lvl]:
            idx = (idx + 1) % 3
            continue

        avail = [i for i in range(len(pool[lvl])) if i not in asked[lvl]]
        if not avail:
            if all(len(asked[l]) == len(pool[l]) for l in levels):
                break
            idx = (idx + 1) % 3
            streak = 0
            continue

        qi = random.choice(avail)
        asked[lvl].add(qi)
        q, opts, ans, exp = pool[lvl][qi]

        user = ask_q(f"[{lvl.upper()}] {q}", opts)
        if user == ans:
            streak += 1
            print("Correct")
            if streak >= 2 and idx < 2:
                idx += 1
                streak = 0
        else:
            print("Wrong")
            if exp:
                print(exp)
            streak = 0
            if idx > 0:
                idx -= 1


This cell acts as the central controller of the system.
It ensures topics are attempted in a structured sequence.
It integrates basic evaluation and adaptive advanced testing.
All system components are coordinated from one place.
This improves readability and debugging.
It represents the complete learning pipeline.

In [ ]:
def main():
    user = input("Enter your name: ").strip()
    if not user:
        return

    data = load_questions_from_csv(QUESTIONS_CSV)
    if not data:
        return

    topics, meta, basics, adv = data

    for i, tid in enumerate(topics, start=1):
        if tid not in basics:
            continue

        score = run_basic_topic(basics[tid], meta[tid], i)

        if score >= TOPIC_PASS_PERCENT and tid in adv and any(adv[tid].values()):
            run_advanced_topic(meta[tid], adv[tid])

    print("\nSession complete.")

main()


This system demonstrates how AI principles can be applied without heavy models.
Adaptivity is achieved through logic, data tracking, and feedback loops.
The design is scalable, modular, and exam-oriented.
Future work can include ML-based recommendation or analytics dashboards.
The project focuses on learning effectiveness, not just automation.
This makes it suitable for real educational use.